In [ ]:
# ---- GLOBAL CONFIG & PARAMS ----
import os, glob, csv, time
from pathlib import Path
import numpy as np

# Paths
DATA_DIR    = "../lab2"   
RESULTS_DIR = "../results"
os.makedirs(RESULTS_DIR, exist_ok=True)

# Reproducibility and RNG
SEED = 42
rng_global = np.random.default_rng(SEED)   # <— THIS defines rng_global

# Greedy/HC/SA knobs
P_RANDOM        = 0.10        # <— THIS defines P_RANDOM (random step prob. in greedy start)
HC_RESTARTS     = 8           # number of random/greedy restarts per instance
HC_MAX_SWEEPS   = 8000        # max sweeps per HC run
HC_STRATEGY     = "first"     # {"first","best"} neighbour acceptance
SEED_METHOD     = "greedy"    # {"greedy","random"} initial tour
PAIR_SAMPLE_CAP = 20000       # None for full neighbourhood; else sample this many pairs
USE_SA          = True        # use simulated annealing inside HC
SA_ALPHA        = 0.995       # cooling factor each sweep

def init_temperature(D: np.ndarray) -> float:
    """A robust initial temperature based on positive median edge length."""
    pos = D[D > 0]
    return float(np.median(pos)) if pos.size else 1.0

print("Ready. DATA_DIR =", DATA_DIR, "| RESULTS_DIR =", RESULTS_DIR)


Ready. DATA_DIR = ../lab2 | RESULTS_DIR = ../results


In [2]:
import numpy as np

def tour_length(D: np.ndarray, tour: np.ndarray) -> float:
    """Closed tour length; works with negative edges too."""
    return float(D[tour, np.roll(tour, -1)].sum())

def greedy_seed(D: np.ndarray, rng: np.random.Generator, p_random: float = 0.10) -> np.ndarray:
    """ε-greedy nearest neighbor to build a feasible permutation."""
    n = D.shape[0]
    start = int(rng.integers(0, n))
    unvisited = set(range(n))
    tour = [start]
    unvisited.remove(start)
    while unvisited:
        last = tour[-1]
        if rng.random() < p_random:
            nxt = int(rng.choice(list(unvisited)))
        else:
            # nearest among unvisited
            cand = list(unvisited)
            costs = D[last, cand]
            nxt = cand[int(np.argmin(costs))]
        tour.append(nxt)
        unvisited.remove(nxt)
    # normalize so city 0 is first (helps 2-opt avoid wrap edge quirks)
    tour = np.asarray(tour, dtype=int)
    i0 = int(np.where(tour == 0)[0][0]) if 0 in tour else 0
    tour = np.roll(tour, -i0)
    return tour

def random_seed(D: np.ndarray, rng: np.random.Generator) -> np.ndarray:
    n = D.shape[0]
    tour = np.arange(n, dtype=int)
    rng.shuffle(tour)
    # normalize to start at 0 if present
    i0 = int(np.where(tour == 0)[0][0]) if 0 in tour else 0
    tour = np.roll(tour, -i0)
    return tour



In [7]:
def two_opt_pass(D, tour, cur_len, rng, strategy="first", pair_sample_cap=None,
                 use_sa=False, temp=0.0, alpha=0.995):
    """
    One 'sweep' over 2-opt moves.
    Returns (improved, tour, new_len, temp)
    """
    n = len(tour)
    if n < 4:                     # <---- new safety guard
        return False, tour, cur_len, temp

    improved_any = False

    if pair_sample_cap is None:
        # full systematic pass
        best_delta = 0.0
        best_pair = None
        for i in range(n - 1):
            a, b = tour[i], tour[(i + 1) % n]
            k_max_excl = (n - 1) if i == 0 else n
            for k in range(i + 2, k_max_excl):
                c, d = tour[k], tour[(k + 1) % n]
                delta = (D[a, c] + D[b, d]) - (D[a, b] + D[c, d])
                accept = (delta < -1e-12) or (use_sa and (rng.random() < np.exp(-delta / max(temp,1e-12))))
                if accept:
                    if strategy == "first":
                        tour[i+1:k+1] = tour[i+1:k+1][::-1]
                        cur_len += delta
                        if use_sa and delta > 0: temp *= alpha
                        return True, tour, cur_len, temp
                    else:
                        if delta < best_delta:
                            best_delta = float(delta)
                            best_pair = (i, k)

        if best_pair is not None:
            i, k = best_pair
            tour[i+1:k+1] = tour[i+1:k+1][::-1]
            cur_len += best_delta
            if use_sa and best_delta > 0:
                temp *= alpha
            improved_any = True
        return improved_any, tour, cur_len, temp

    else:
        # stochastic pass
        m = pair_sample_cap
        for _ in range(m):
            i = int(rng.integers(0, n - 2))     # <---- fix: must allow k room
            low = i + 2
            hi  = (n - 1) if i == 0 else n      # hi exclusive
            if low >= hi:
                continue
            k = int(rng.integers(low, hi))

            a, b = tour[i], tour[(i + 1) % n]
            c, d = tour[k], tour[(k + 1) % n]
            delta = (D[a, c] + D[b, d]) - (D[a, b] + D[c, d])
            accept = (delta < -1e-12) or (use_sa and (rng.random() < np.exp(-delta / max(temp,1e-12))))
            if accept:
                tour[i+1:k+1] = tour[i+1:k+1][::-1]
                cur_len += delta
                if use_sa and delta > 0: temp *= alpha
                if strategy == "first":
                    return True, tour, cur_len, temp
                else:
                    improved_any = True

        return improved_any, tour, cur_len, temp


def hill_climb_once(D, rng, seed_method="greedy", p_random=0.10,
                    max_sweeps=50, strategy="first", pair_sample_cap=None,
                    use_sa=False, t0=0.0, sa_alpha=0.995):
    if seed_method == "greedy":
        tour = greedy_seed(D, rng, p_random=p_random)
    else:
        tour = random_seed(D, rng)
    cur_len = tour_length(D, tour)
    temp = float(t0)
    sweeps = 0
    while sweeps < max_sweeps:
        sweeps += 1
        improved, tour, cur_len, temp = two_opt_pass(
            D, tour, cur_len, rng,
            strategy=strategy,
            pair_sample_cap=pair_sample_cap,
            use_sa=use_sa,
            temp=temp,
            alpha=sa_alpha
        )
        if not improved:
            break
    return tour, cur_len, sweeps


In [8]:
import os, glob, csv, time

out_csv = os.path.join(RESULTS_DIR, "hc_results.csv")
rows = []
files = sorted(glob.glob(os.path.join(DATA_DIR, "*.npy")))

for f in files:
    D = np.load(f)
    n = D.shape[0]
    t0 = time.time()

    best_len = np.inf
    best_tour = None
    total_sweeps = 0

    # SA initial temp (if used)
    T0 = init_temperature(D) if USE_SA else 0.0

    for r in range(HC_RESTARTS):
        tour, length, sweeps = hill_climb_once(
            D, rng_global,
            seed_method=SEED_METHOD,
            p_random=P_RANDOM,
            max_sweeps=HC_MAX_SWEEPS,
            strategy=HC_STRATEGY,
            pair_sample_cap=PAIR_SAMPLE_CAP,
            use_sa=USE_SA,
            t0=T0,
            sa_alpha=SA_ALPHA
        )
        total_sweeps += sweeps
        if length < best_len:
            best_len = length
            best_tour = tour.copy()

    elapsed = time.time() - t0
    rows.append({
        "file": os.path.basename(f),
        "n": n,
        "best_cost": best_len,
        "restarts": HC_RESTARTS,
        "avg_sweeps": round(total_sweeps / HC_RESTARTS, 2),
        "strategy": HC_STRATEGY,
        "seed": SEED_METHOD,
        "pair_sample_cap": PAIR_SAMPLE_CAP if PAIR_SAMPLE_CAP is not None else "full",
        "use_sa": USE_SA,
        "seconds": round(elapsed, 3),
    })
    print(f"[HC] {os.path.basename(f):>18} | n={n:4d} | cost={best_len:12.3f} | "
          f"sweeps~{total_sweeps/HC_RESTARTS:.1f} | {elapsed:.2f}s")

# write CSV
with open(out_csv, "w", newline="") as fp:
    writer = csv.DictWriter(fp, fieldnames=list(rows[0].keys()))
    writer.writeheader()
    writer.writerows(rows)

print("\nSaved:", out_csv)


[HC]   problem_g_10.npy | n=  10 | cost=    1497.664 | sweeps~1270.0 | 10.36s
[HC]  problem_g_100.npy | n= 100 | cost=    4017.487 | sweeps~2128.6 | 18.53s
[HC] problem_g_1000.npy | n=1000 | cost=   13180.080 | sweeps~4786.5 | 53.77s
[HC]   problem_g_20.npy | n=  20 | cost=    1755.515 | sweeps~2667.2 | 23.74s
[HC]  problem_g_200.npy | n= 200 | cost=    5663.379 | sweeps~2744.4 | 43.10s
[HC]   problem_g_50.npy | n=  50 | cost=    2663.122 | sweeps~2510.4 | 38.02s
[HC]  problem_g_500.npy | n= 500 | cost=    9162.392 | sweeps~3635.5 | 42.81s
[HC]  problem_r1_10.npy | n=  10 | cost=  -12950.398 | sweeps~2146.4 | 2.35s
[HC] problem_r1_100.npy | n= 100 | cost=  -19115.273 | sweeps~4824.1 | 99.69s
[HC] problem_r1_1000.npy | n=1000 | cost=  -43271.693 | sweeps~8000.0 | 14.32s
[HC]  problem_r1_20.npy | n=  20 | cost=  -10766.837 | sweeps~2837.0 | 15.95s
[HC] problem_r1_200.npy | n= 200 | cost=  -38118.556 | sweeps~7652.5 | 42.83s
[HC]  problem_r1_50.npy | n=  50 | cost=  -17925.004 | sweeps~37